In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import pandas as pd
import os

# Import your CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))

if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare Logo
image_filename = 'Grazioso Salvare Logo.png'
try:
    with open(image_filename, 'rb') as f:
        encoded_image = base64.b64encode(f.read()).decode()
except FileNotFoundError:
    print("Logo file not found.")
    encoded_image = ""

app.layout = html.Div([
    html.Center([
        html.Img(src=f'data:image/png;base64,{encoded_image}', style={'height':'100px'}) if encoded_image else html.Div(),
        html.B(html.H1('CS-340 Dashboard')),
        html.P("Developed by Niaz Khan", style={'fontSize': '18px'})
    ]),
    html.Hr(),

    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        labelStyle={'display': 'inline-block', 'margin-right': '20px'}
    ),
    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        row_selectable='single',
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={
            'minWidth': '120px', 'width': '150px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        },
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'}
    ),

    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ]),

    html.Hr(),
    html.Pre(id='debug-output', style={'fontSize': '12px', 'color': '#555'})
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    rescue_breeds = {
        "Water Rescue": ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"],
        "Mountain or Wilderness Rescue": ["German Shepherd", "Alaskan Malamute", "Border Collie"],
        "Disaster or Individual Tracking": ["Bloodhound", "Rottweiler", "Doberman Pinscher"]
    }

    if filter_type == 'Reset':
        query = {}
    else:
        query = {
            "breed": {"$in": rescue_breeds[filter_type]},
            "age_upon_outcome_in_weeks": {"$lte": 104}
        }

    filtered_df = pd.DataFrame(list(db.read(query)))
    if '_id' in filtered_df.columns:
        filtered_df.drop(columns=['_id'], inplace=True)

    essential_fields = ['name', 'breed', 'location_lat', 'location_long']
    filtered_df.dropna(subset=essential_fields, inplace=True)

    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame(viewData)
    if dff.empty or 'breed' not in dff.columns:
        return [html.P("No data to display.")]
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if not viewData:
        return [html.P("No map data to display.")]

    dff = pd.DataFrame.from_dict(viewData)
    if dff.empty or 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return [html.P("Latitude/Longitude data not available.")]

    row = index[0] if index else 0
    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']
    breed = dff.iloc[row].get('breed', 'Breed Unknown')
    name = dff.iloc[row].get('name', 'Unnamed')

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]

@app.callback(
    Output('debug-output', 'children'),
    [Input('datatable-id', 'data')]
)
def debug_output(data):
    if data and len(data) > 0:
        return f"Sample record:\n{data[0]}"
    else:
        return "No data loaded."

# Run the dashboard (use a different port to avoid conflict)
app.run_server(debug=True, port=8051)

Dash app running on http://127.0.0.1:8051/
